In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

pd.options.display.max_rows = 10
pd.options.display.max_columns = 15

guest_id = int(input('insert a user id : '))

insert a user id : 120


In [2]:
data = pd.read_table('data/u.data', sep='\t', usecols=['user id', 'movie id', 'rating'])
data

,user id,movie id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [3]:
data = data.pivot_table(index='movie id', columns='user id', values='rating', fill_value=0)
data

user id,1,2,3,4,5,6,7,...,937,938,939,940,941,942,943
movie id,,,,,,,,,,,,,,,
1,5,4,0,0,4,4,0,...,0,4,0,0,5,0,0
2,3,0,0,0,3,0,0,...,0,0,0,0,0,0,5
3,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0
4,3,0,0,0,0,0,5,...,0,0,0,2,0,0,0
5,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0
1679,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0
1680,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0


當余弦值越大則余弦夾角越小，表示兩向量越相似；反之，余弦值越小則余弦夾角越大，表示兩向量差異越大。
余弦值 = 1 時，余弦夾角為 0 度。
余弦值 = 0 時，余弦夾角為 90 度。
余弦值 = -1 時，兩向量為相反方向。

In [4]:
sim = DataFrame()
for i in data.columns:
    sim[i] = DataFrame(cosine_similarity([data[guest_id]], [data[i]]))

sim

,1,2,3,4,5,6,7,...,937,938,939,940,941,942,943
0,0.18461,0.25678,0.030739,0.109857,0.10696,0.152293,0.13165,...,0.27807,0.394923,0.342343,0.110807,0.215339,0.098953,0.155048


In [5]:
plt.scatter(sim, sim, s=5, alpha=0.5)
plt.show()

In [6]:
top10_near = sim.T.sort_values(by=0, ascending=False).head(11).T
top10_near

,120,674,549,68,935,706,637,66,634,800,26
0,1.0,0.512915,0.508351,0.504689,0.498027,0.467133,0.456705,0.455357,0.454038,0.447082,0.445753


In [7]:
drop_df = data[data[guest_id] > 0][guest_id].to_frame('rating').T
drop_df

movie id,1,9,15,25,50,117,118,...,508,515,546,742,744,827,924
rating,4,4,4,5,4,3,2,...,2,5,2,4,4,2,4


In [8]:
mult_rec = DataFrame()
for i in top10_near.columns:
    if i != guest_id:
        mult_rec[i] = data[i] * (1 + top10_near[i][0])

data = mult_rec.drop(drop_df)
data

,674,549,68,935,706,637,66,634,800,26
movie id,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
data = data.T.sum().sort_values(ascending=False).to_frame('recommend rate').head(10)
data

,recommend rate
movie id,
181,56.144096
300,38.111206
288,36.958861
294,36.606393
471,35.272563
100,30.866353
313,29.553661
597,29.392909
111,26.602167


In [10]:
item = pd.read_table('data/u.item', sep='|', usecols=['movie id', 'movie title', 'release date'])
item

,movie id,movie title,release date
0,1,Toy Story (1995),01-Jan-1995
1,2,GoldenEye (1995),01-Jan-1995
2,3,Four Rooms (1995),01-Jan-1995
3,4,Get Shorty (1995),01-Jan-1995
4,5,Copycat (1995),01-Jan-1995
...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998
1678,1679,B. Monkey (1998),06-Feb-1998
1679,1680,Sliding Doors (1998),01-Jan-1998
1680,1681,You So Crazy (1994),01-Jan-1994


In [11]:
item = item.set_index(['movie id'])
item

,movie title,release date
movie id,,
1,Toy Story (1995),01-Jan-1995
2,GoldenEye (1995),01-Jan-1995
3,Four Rooms (1995),01-Jan-1995
4,Get Shorty (1995),01-Jan-1995
5,Copycat (1995),01-Jan-1995
...,...,...
1678,Mat' i syn (1997),06-Feb-1998
1679,B. Monkey (1998),06-Feb-1998
1680,Sliding Doors (1998),01-Jan-1998


In [12]:
pd.concat([data, item], axis=1, join_axes=[data.index])

,recommend rate,movie title,release date
movie id,,,
181,56.144096,Return of the Jedi (1983),14-Mar-1997
300,38.111206,Air Force One (1997),01-Jan-1997
288,36.958861,Scream (1996),20-Dec-1996
294,36.606393,Liar Liar (1997),21-Mar-1997
471,35.272563,Courage Under Fire (1996),08-Mar-1996
100,30.866353,Fargo (1996),14-Feb-1997
313,29.553661,Titanic (1997),01-Jan-1997
597,29.392909,Eraser (1996),21-Jun-1996
111,26.602167,"Truth About Cats & Dogs, The (1996)",26-Apr-1996
